In [1]:
# Alan's code from Map

# import libraries and read data
import pandas as pd
import numpy as np
from arcgis.gis import *
from ast import literal_eval

df = pd.read_csv('Crime_Data_from_2010_to_Present.csv')

# pick out bad entries nan in 'Location'
temp = [p for p in df['Location ']]
badentries = []
for i in range(len(temp)):
    if type(temp[i]) != str:
        badentries.append(i)
badentries #ones with location nan

[659026, 736132, 1473914, 1532070, 1532072, 1532073, 1532086, 1532087, 1532089]

In [2]:
# pick out bad entries '(0,0)' in 'Location'
df1 = df.drop(df.index[badentries])
#df1 = df.drop(badentries)
temp = [p for p in df1['Location ']]
for i in range(len(temp)):
    if type(temp[i]) != str:
        print(i)
        
df1['Location '] = df1['Location '].apply(literal_eval) 

In [3]:
grp =df.groupby('Location ')
Nan = grp.get_group('(0, 0)')# "No location"
Nan.count()
df1 = df1.drop(Nan.index)

In [4]:
# Grouping crimes together

important_crime_types = ['ASSAULT','HOMICIDE','WEAPONS','SEXUAL OFFENSE',
                         'CRIMES AGAINST CHILDREN','THEFT','VEHICLE']

Assault = ['INTIMATE PARTNER - SIMPLE ASSAULT','BATTERY - SIMPLE ASSAULT',
           'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT',
           'CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT', 'OTHER ASSAULT',
           'INTIMATE PARTNER - AGGRAVATED ASSAULT', 
           'CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT',
           'ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER','BATTERY POLICE (SIMPLE)',
           'BATTERY ON A FIREFIGHTER']

Homicide = ['CRIMINAL HOMICIDE','LYNCHING - ATTEMPTED','LYNCHING',
            'MANSLAUGHTER', 'NEGLIGENT']

Weapons = ['BOMB SCARE','SHOTS FIRED AT INHABITED DWELLING',
           'BRANDISH WEAPON','ARSON','DISCHARGE FIREARMS/SHOTS FIRED',
           'WEAPONS POSSESSION/BOMBING',
           'REPLICA FIREARMS(SALE,DISPLAY,MANUFACTURE OR DISTRIBUTE)0132',
           'SHOTS FIRED AT MOVING VEHICLE, TRAIN OR AIRCRAFT',
           'FIREARMS RESTRAINING ORDER (FIREARMS RO)']

Sexual_Offense = ['SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH 0007=02',
                  'INDECENT EXPOSURE','RAPE, FORCIBLE',
                  'BATTERY WITH SEXUAL CONTACT',
                  'SEXUAL PENTRATION WITH A FOREIGN OBJECT','LETTERS, LEWD',
                  'RAPE, ATTEMPTED','LEWD CONDUCT','PIMPING','SEX, UNLAWFUL',
                  'ORAL COPULATION','PANDERING',
                  'INCEST (SEXUAL ACTS BETWEEN BLOOD RELATIVES)']

Crimes_against_Children = ['CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT',
                           'CHILD NEGLECT (SEE 300 W.I.C.)',
                           'CHILD ANNOYING (17YRS & UNDER)',
                           'CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 YRS OLDER)0060',
                           'CHILD STEALING','CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT',
                           'CHILD ABANDONMENT','DRUGS, TO A MINOR']

Theft = ['THEFT PLAIN - PETTY ($950 & UNDER)',
         'SHOPLIFTING - PETTY THEFT ($950 & UNDER)',
         'EMBEZZLEMENT, PETTY THEFT ($950 & UNDER)',
         'THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD0036',
         'DEFRAUDING INNKEEPER/THEFT OF SERVICES, $400 & UNDER',
         'SHOPLIFTING-GRAND THEFT ($950.01 & OVER)',
         'THEFT, PERSON','THEFT PLAIN - ATTEMPT',
         'THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER)','THEFT OF IDENTITY',
         'BUNCO, GRAND THEFT','BUNCO, PETTY THEFT','THEFT FROM PERSON - ATTEMPT',
         'DISHONEST EMPLOYEE - PETTY THEFT','THEFT FROM MOTOR VEHICLE - ATTEMPT',
         'DISHONEST EMPLOYEE - GRAND THEFT',
         'DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $400',
         'GRAND THEFT / INSURANCE FRAUD',
         'THEFT, COIN MACHINE - GRAND ($950.01 & OVER)',
         'GRAND THEFT / AUTO REPAIR','PETTY THEFT - AUTO REPAIR',
         'TILL TAP - GRAND THEFT ($950.01 & OVER)','THEFT, COIN MACHINE - ATTEMPT']

Vehicle = ['VEHICLE - ATTEMPT STOLEN',
           'VEHICLE - STOLEN', 
           'BURGLARY FROM VEHICLE', 
           'THEFT FROM MOTOR VEHICLE - ATTEMPT', 
           'THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)', 
           'THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER)', 
           'THROWING OBJECT AT MOVING VEHICLE',
           'BURGLARY FROM VEHICLE, ATTEMPTED']

Other = Assault + Homicide + Crimes_against_Children

In [5]:
# reset coulmn names to all lower cases
df1.columns = ['DR_Number','date_reported','date_occurred','time_occurred','area_ID','area_name','reporting_district','crime_code','crime_code_description','MO_code','victim_age','victim_sex','victim_descent',	'premise_code',	'premise_description','weapon_used_code','weapon_description','status_code','status_description','crime_code1','crime_code2','crime_code3','crime_code4','address','cross_street','location']

In [6]:
# construct 'x' and 'y' and delete 'location'

new_col_list = ['y','x']
for n,col in enumerate(new_col_list):
    df1[col] = df1['location'].apply(lambda location: location[n])
del df1['location']

In [7]:
df1.columns

Index(['DR_Number', 'date_reported', 'date_occurred', 'time_occurred',
       'area_ID', 'area_name', 'reporting_district', 'crime_code',
       'crime_code_description', 'MO_code', 'victim_age', 'victim_sex',
       'victim_descent', 'premise_code', 'premise_description',
       'weapon_used_code', 'weapon_description', 'status_code',
       'status_description', 'crime_code1', 'crime_code2', 'crime_code3',
       'crime_code4', 'address', 'cross_street', 'y', 'x'],
      dtype='object')

In [8]:
# construct SpatialDataFrame with DR Number, Crime code desc, data occurred, Shape

df2 = pd.DataFrame({'DR_Number':df1['DR_Number'],
                    'crime_code_description': df1["crime_code_description"],
                    'date_occurred': df1["date_occurred"],
                    'SHAPE':[0]*len(df1['DR_Number'])})
df2['SHAPE'] = df1.apply(lambda row : arcgis.geometry.Geometry({'x': row['x'], 'y': row['y']}), axis=1 )

In [9]:
# Assigning a single label for each group of crimes

#df2.loc[(df2.crime_code_description.isin(Other)),'crime_code_description'] = 'OTHER'
df2.loc[(df2.crime_code_description.isin(Assault)),'crime_code_description'] = 'ASSAULT'
df2.loc[(df2.crime_code_description.isin(Homicide)),'crime_code_description'] = 'HOMICIDE'
df2.loc[(df2.crime_code_description.isin(Crimes_against_Children)),'crime_code_description'] = 'CHILDREN'
df2.loc[(df2.crime_code_description.isin(Weapons)),'crime_code_description'] = 'WEAPONS'
df2.loc[(df2.crime_code_description.isin(Sexual_Offense)),'crime_code_description'] = 'SEXUAL OFFENSE'
df2.loc[(df2.crime_code_description.isin(Theft)),'crime_code_description'] = 'THEFT'
df2.loc[(df2.crime_code_description.isin(Vehicle)),'crime_code_description'] = 'VEHICLE'

In [10]:
#in16_17 = df1[(df1.date_occurred.str.endswith('16')) | (df1.date_occurred.str.endswith('17'))]

In [12]:
# dataframes for each big category of crimes, and important crimes

df_important = df2[df2['crime_code_description'].isin(important_crime_types)]
df_OTHER = df2[(df2['crime_code_description'] == 'ASSAULT') | (df2['crime_code_description'] == 'HOMICIDE') |
                  (df2['crime_code_description'] == 'CHILDREN')]
df_ASSAULT = df2[df2['crime_code_description'] == 'ASSAULT']
df_HOMICIDE = df2[df2['crime_code_description'] == 'HOMICIDE']
df_CHILDREN = df2[df2['crime_code_description'] == 'CHILDREN']
df_WEAPONS = df2[df2['crime_code_description'] == 'WEAPONS']
df_SEXUAL = df2[df2['crime_code_description'] == 'SEXUAL OFFENSE']
df_THEFT = df2[df2['crime_code_description'] == 'THEFT']
df_VEHICLE = df2[df2['crime_code_description'] == 'VEHICLE']

In [13]:
# dataframes for each big category of crimes, and important crimes, in 2016 and 2017

df_important_16_17 = df_important[(df_important.date_occurred.str.endswith('16')) | (df_important.date_occurred.str.endswith('17'))]
df_OTHER_16_17 = df_OTHER[(df_OTHER.date_occurred.str.endswith('16')) | (df_OTHER.date_occurred.str.endswith('17'))]
df_ASSAULT_16_17 = df_ASSAULT[(df_ASSAULT.date_occurred.str.endswith('16')) | (df_ASSAULT.date_occurred.str.endswith('17'))]
df_HOMICIDE_16_17 = df_HOMICIDE[(df_HOMICIDE.date_occurred.str.endswith('16')) | (df_HOMICIDE.date_occurred.str.endswith('17'))]
df_CHILDREN_16_17 = df_CHILDREN[(df_CHILDREN.date_occurred.str.endswith('16')) | (df_CHILDREN.date_occurred.str.endswith('17'))]
df_WEAPONS_16_17 = df_WEAPONS[(df_WEAPONS.date_occurred.str.endswith('16')) | (df_WEAPONS.date_occurred.str.endswith('17'))]
df_SEXUAL_16_17 = df_SEXUAL[(df_SEXUAL.date_occurred.str.endswith('16')) | (df_SEXUAL.date_occurred.str.endswith('17'))]
df_THEFT_16_17 = df_THEFT[(df_THEFT.date_occurred.str.endswith('16')) | (df_THEFT.date_occurred.str.endswith('17'))]
df_VEHICLE_16_17 = df_VEHICLE[(df_VEHICLE.date_occurred.str.endswith('16')) | (df_VEHICLE.date_occurred.str.endswith('17'))]

In [14]:
# convert df to sdf

from arcgis import SpatialDataFrame

df_important_16_17_sdf = SpatialDataFrame(df_important_16_17)
df_OTHER_16_17_sdf = SpatialDataFrame(df_OTHER_16_17)
df_ASSAULT_16_17_sdf = SpatialDataFrame(df_ASSAULT_16_17)
df_HOMICIDE_16_17_sdf = SpatialDataFrame(df_HOMICIDE_16_17)
df_CHILDREN_16_17_sdf = SpatialDataFrame(df_CHILDREN_16_17)
df_WEAPONS_16_17_sdf = SpatialDataFrame(df_WEAPONS_16_17)
df_SEXUAL_16_17_sdf = SpatialDataFrame(df_SEXUAL_16_17)
df_THEFT_16_17_sdf = SpatialDataFrame(df_THEFT_16_17)
df_VEHICLE_16_17_sdf = SpatialDataFrame(df_VEHICLE_16_17)

In [15]:
df_OTHER_16_17_sdf

,DR_Number,SHAPE,crime_code_description,date_occurred
1208239,151601466,"{'x': -118.4238, 'y': 34.2606}",ASSAULT,01/01/2016
1208240,151601467,"{'x': -118.2818, 'y': 34.2463}",ASSAULT,01/02/2016
1208241,151801503,"{'x': -118.2358, 'y': 33.9431}",ASSAULT,01/01/2016
1208243,160100503,"{'x': -118.254, 'y': 34.0383}",ASSAULT,01/01/2016
1208244,160100504,"{'x': -118.2418, 'y': 34.0493}",ASSAULT,01/01/2016
1208248,160100509,"{'x': -118.2476, 'y': 34.0461}",ASSAULT,01/03/2016
1208250,160100511,"{'x': -118.2414, 'y': 34.0491}",ASSAULT,01/02/2016
1208251,160100513,"{'x': -118.2455, 'y': 34.0555}",ASSAULT,01/03/2016
1208252,160100514,"{'x': -118.2499, 'y': 34.0453}",ASSAULT,01/06/2016
1208256,160100520,"{'x': -118.2415, 'y': 34.0449}",ASSAULT,01/08/2016


In [18]:
df_CHILDREN_16_17_sdf

,DR_Number,SHAPE,crime_code_description,date_occurred
1208427,160100923,"{'x': -118.242, 'y': 34.043}",CHILDREN,05/24/2016
1208446,160100963,"{'x': -118.2439, 'y': 34.0442}",CHILDREN,06/06/2016
1208468,160101004,"{'x': -118.2468, 'y': 34.0652}",CHILDREN,06/20/2016
1208817,160104275,"{'x': -118.2492, 'y': 34.0652}",CHILDREN,01/02/2016
1209301,160105973,"{'x': -118.2439, 'y': 34.0442}",CHILDREN,01/22/2016
1209733,160107404,"{'x': -118.2716, 'y': 34.0381}",CHILDREN,02/10/2016
1210409,160109551,"{'x': -118.2455, 'y': 34.0437}",CHILDREN,03/09/2016
1210511,160109802,"{'x': -118.2381, 'y': 34.049}",CHILDREN,03/14/2016
1212682,160116005,"{'x': -118.2556, 'y': 34.0467}",CHILDREN,05/31/2016
1212843,160116469,"{'x': -118.2378, 'y': 34.0583}",CHILDREN,05/30/2016


In [19]:
# To-Do Dear Alan, please create a layer for each sdf created, which are:
# df_important_16_17_sdf
# df_OTHER_16_17_sdf
# df_ASSAULT_16_17_sdf
# df_HOMICIDE_16_17_sdf
# df_CHILDREN_16_17_sdf
# df_WEAPONS_16_17_sdf
# df_SEXUAL_16_17_sdf
# df_THEFT_16_17_sdf
# df_VEHICLE_16_17_sdf